<a href="https://colab.research.google.com/github/kaisarmasum/Quantum-Machine-Learning/blob/main/Text_Classification_VQC%2BSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(32)

from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import dask.dataframe as dd
#df =  pd.read_csv("Twitter.csv",sep="\t")
df = pd.read_csv("spamdata.csv")
#df = pd.read_fwf("/content/sst2.txt", sep=" ", header=None)
#df = pd.read_csv("IMDB.csv")
#df = dd.read_csv("Yelp.csv")
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#df = pd.DataFrame(df)
#df = df[:6000]
df.columns = ["label", "text"]
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
len(df)

5572

In [ ]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
df.label= l.fit_transform(df.label)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def lemmatize(text):

   doc = nlp(text)

   tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]

   return ' '.join(tokens)

In [ ]:
df['processed_text'] = df.text.apply(lambda txt : lemmatize(txt))

In [ ]:
df.head()

,label,text,processed_text
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win FA Cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun early hor u c
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think go usf live


In [ ]:
df.isnull().sum()

label             0
text              0
processed_text    0
dtype: int64

In [ ]:
X = df['processed_text']
y = df.label

In [ ]:
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=8000)
Tfidf_vect.fit(df['processed_text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  97.75784753363229


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Test_Y, predictions_SVM ))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       969
           1       0.98      0.85      0.91       146

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=100)
clf.fit(Train_X_Tfidf, Train_Y)
predictions_MLP = clf.predict(Test_X_Tfidf)
print("MLP Accuracy Score -> ",accuracy_score(predictions_MLP, Test_Y)*100)

MLP Accuracy Score ->  97.9372197309417


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Test_Y, predictions_MLP))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       969
           1       0.97      0.87      0.92       146

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
# Get the number of parameters
num_params = sum(len(v) if isinstance(v, list) else 1 for k, v in clf.get_params(deep=True).items())

print("Number of Parameters in MLPClassifier:", num_params)

Number of Parameters in MLPClassifier: 23
